# Day11_2 별첨: 연관 분석 (Association Rule Mining)

## 📚 학습 목표

**Part 1: 연관 규칙 기초**
1. 연관 분석의 개념과 비즈니스 활용 이해하기
2. 지지도(Support), 신뢰도(Confidence), 향상도(Lift) 이해하기
3. TransactionEncoder로 데이터 변환하기
4. Apriori 알고리즘으로 빈발 항목집합 찾기
5. 연관 규칙 생성 및 해석하기

**Part 2: 연관 분석 심화 및 응용**
1. 추가 지표 (Conviction, Leverage, Zhang's Metric) 이해하기
2. Plotly로 연관 규칙 시각화하기
3. 추천 시스템 함수 구현하기
4. Gradio UI로 인터랙티브 데모 만들기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 비즈니스 활용 | 예시 |
|------|-------------|------|
| 장바구니 분석 | 상품 배치 최적화 | "맥주 옆에 기저귀 배치" |
| 교차 판매 | 매출 증대 | "이 상품을 산 고객이 함께 산..." |
| 번들 상품 | 프로모션 기획 | "라면 + 김치 세트 할인" |
| 키워드 연관 | 태그 추천 | 리뷰에서 연관 키워드 추출 |

**분석가 관점**: 연관 분석은 수백만 건의 거래에서 숨겨진 패턴을 찾아내는 강력한 도구입니다!

---

# Part 1: 연관 규칙 기초

---

## 1.1 연관 분석이란?

### 정의

**연관 분석(Association Rule Mining)**은 대규모 데이터에서 항목 간의 관계(규칙)를 발견하는 데이터 마이닝 기법입니다.

### 대표 사례: 맥주와 기저귀

1990년대 월마트의 분석 결과:
- 금요일 저녁, 기저귀를 사는 고객이 맥주도 함께 구매
- 이유: 주말 대비 아기 용품을 사러 온 아빠들이 맥주도 구매
- 대응: 맥주와 기저귀를 가까이 배치 → 매출 증가

### 연관 규칙의 형태

```
{기저귀} → {맥주}
```

- **전제(Antecedent)**: 기저귀
- **결론(Consequent)**: 맥주
- 해석: "기저귀를 사면 맥주도 살 가능성이 높다"

In [ ]:
# 필요한 라이브러리 설치 (최초 1회)
# !pip install mlxtend pandas plotly gradio

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# mlxtend (연관 분석)
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

print("라이브러리 임포트 완료")

---

## 1.2 핵심 지표: 지지도, 신뢰도, 향상도

### 1. 지지도 (Support)

**정의**: 전체 거래 중 해당 항목(집합)이 포함된 비율

$$Support(A) = \frac{A를 포함한 거래 수}{전체 거래 수}$$

$$Support(A \rightarrow B) = \frac{A와 B를 모두 포함한 거래 수}{전체 거래 수}$$

### 2. 신뢰도 (Confidence)

**정의**: A를 포함한 거래 중 B도 포함한 비율

$$Confidence(A \rightarrow B) = \frac{Support(A \cup B)}{Support(A)}$$

### 3. 향상도 (Lift)

**정의**: A와 B가 독립일 때 대비 실제 동시 발생 비율

$$Lift(A \rightarrow B) = \frac{Confidence(A \rightarrow B)}{Support(B)} = \frac{Support(A \cup B)}{Support(A) \times Support(B)}$$

- **Lift > 1**: A와 B는 양의 상관관계 (함께 구매 경향)
- **Lift = 1**: A와 B는 독립 (관련 없음)
- **Lift < 1**: A와 B는 음의 상관관계 (함께 구매 안 함)

### 💡 실무 예시: 지표 수동 계산

In [ ]:
# 간단한 예시 데이터
simple_transactions = [
    ['우유', '빵'],
    ['우유', '빵', '버터'],
    ['우유', '계란'],
    ['빵', '버터'],
    ['우유', '빵', '버터', '계란']
]

total = len(simple_transactions)  # 5
print(f"총 거래 수: {total}")

# 항목별 등장 횟수
from collections import Counter

item_counts = Counter()
for trans in simple_transactions:
    item_counts.update(trans)

print(f"\n항목별 등장 횟수:")
for item, count in item_counts.most_common():
    print(f"  {item}: {count}회 (Support: {count/total:.2f})")

In [ ]:
# 수동 계산: {우유} → {빵}
# 우유 포함 거래: 4개
# 우유+빵 포함 거래: 3개
# 빵 포함 거래: 4개

support_milk = 4 / 5  # 0.8
support_bread = 4 / 5  # 0.8
support_milk_bread = 3 / 5  # 0.6

confidence = support_milk_bread / support_milk  # 0.6 / 0.8 = 0.75
lift = confidence / support_bread  # 0.75 / 0.8 = 0.9375

print("규칙: {우유} → {빵}")
print(f"  Support(우유): {support_milk:.3f}")
print(f"  Support(빵): {support_bread:.3f}")
print(f"  Support(우유, 빵): {support_milk_bread:.3f}")
print(f"  Confidence: {confidence:.3f}")
print(f"  Lift: {lift:.3f}")
print(f"\n해석: Lift < 1이므로 우유와 빵은 독립적이거나 약간 음의 상관관계")

---

## 1.3 TransactionEncoder로 데이터 변환

### 원-핫 인코딩 (One-Hot Encoding)

Apriori 알고리즘은 **이진 행렬** 형태의 데이터가 필요합니다.

```
거래 목록:                이진 행렬:
[['우유', '빵'],          우유  빵  버터  계란
 ['우유', '빵', '버터']]   True True False False
                          True True True  False
```

In [ ]:
# TransactionEncoder 사용법
te = TransactionEncoder()
te_array = te.fit_transform(simple_transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

print("원본 거래 데이터:")
for i, trans in enumerate(simple_transactions):
    print(f"  거래 {i+1}: {trans}")

print("\n변환된 이진 행렬:")
print(df_encoded)

---

## 1.4 Apriori 알고리즘 실행

### Apriori 알고리즘이란?

- **목적**: 빈발 항목집합(Frequent Itemsets) 탐색
- **원리**: "빈발 항목의 부분집합도 빈발하다" (Anti-monotone property)
- **파라미터**: `min_support` (최소 지지도 임계값)

In [ ]:
# Apriori 실행
frequent_itemsets = apriori(
    df_encoded,
    min_support=0.4,  # 40% 이상 등장하는 항목만
    use_colnames=True  # 컬럼명 사용
)

print("빈발 항목집합 (min_support=0.4):")
print(frequent_itemsets.sort_values('support', ascending=False))

---

## 1.5 연관 규칙 생성 및 해석

In [ ]:
# 연관 규칙 생성
rules = association_rules(
    frequent_itemsets,
    metric="lift",
    min_threshold=0.5  # Lift >= 0.5인 규칙만
)

print(f"생성된 연관 규칙: {len(rules)}개")
print("\n규칙 미리보기:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

In [ ]:
# 규칙 해석 함수
def interpret_rule(row):
    """연관 규칙 해석"""
    ant = list(row['antecedents'])
    con = list(row['consequents'])
    
    print(f"규칙: {ant} → {con}")
    print(f"  지지도: {row['support']:.3f} ({row['support']*100:.1f}%)")
    print(f"  신뢰도: {row['confidence']:.3f} ({row['confidence']*100:.1f}%)")
    print(f"  향상도: {row['lift']:.3f}")
    
    if row['lift'] > 1:
        print(f"  → 양의 상관관계: {ant}를 사면 {con}도 살 가능성 높음")
    elif row['lift'] < 1:
        print(f"  → 음의 상관관계: {ant}와 {con}는 독립적")
    else:
        print(f"  → 독립: {ant}와 {con}는 관련 없음")

# 상위 규칙 해석
print("상위 연관 규칙 해석")
print("=" * 60)

for i, row in rules.nlargest(3, 'lift').iterrows():
    interpret_rule(row)
    print()

---

### 💡 실무 예시: 한국어 편의점 데이터

In [ ]:
# 한국 편의점 장바구니 데이터 (시뮬레이션)
convenience_store_transactions = [
    ['삼각김밥', '컵라면', '콜라'],
    ['우유', '식빵', '계란'],
    ['맥주', '치킨', '콜라'],
    ['삼각김밥', '커피', '초콜릿'],
    ['라면', '김치', '소주'],
    ['도시락', '콜라', '껌'],
    ['삼각김밥', '컵라면', '이온음료'],
    ['맥주', '과자', '소주'],
    ['우유', '시리얼', '바나나'],
    ['삼각김밥', '콜라', '과자'],
    ['라면', '계란', '김치'],
    ['커피', '샌드위치', '과자'],
    ['맥주', '치킨', '소주'],
    ['삼각김밥', '이온음료', '초콜릿'],
    ['도시락', '콜라', '과자'],
    ['컵라면', '삼각김밥', '콜라'],
    ['맥주', '과자', '콜라'],
    ['커피', '샌드위치', '우유'],
    ['라면', '김치', '계란'],
    ['삼각김밥', '컵라면', '과자'],
    ['맥주', '치킨', '과자'],
    ['우유', '식빵', '잼'],
    ['도시락', '이온음료', '껌'],
    ['삼각김밥', '콜라', '커피'],
    ['라면', '계란', '소주'],
    ['맥주', '소주', '과자'],
    ['커피', '과자', '초콜릿'],
    ['삼각김밥', '컵라면', '콜라', '과자'],
    ['치킨', '맥주', '콜라'],
    ['우유', '시리얼', '과자'],
    ['도시락', '콜라', '이온음료'],
    ['삼각김밥', '이온음료', '과자'],
    ['라면', '김치', '맥주'],
    ['커피', '샌드위치', '초콜릿'],
    ['맥주', '치킨', '과자', '소주'],
    ['삼각김밥', '컵라면', '이온음료', '과자'],
    ['우유', '계란', '식빵'],
    ['콜라', '과자', '껌'],
    ['도시락', '커피', '콜라'],
    ['삼각김밥', '콜라', '과자', '껌']
]

print(f"총 거래 수: {len(convenience_store_transactions)}")

# 상품 빈도 확인
all_items = [item for trans in convenience_store_transactions for item in trans]
item_freq = pd.Series(all_items).value_counts()

print("\n상위 10개 상품:")
print(item_freq.head(10))

In [ ]:
# 데이터 변환
te = TransactionEncoder()
te_array = te.fit_transform(convenience_store_transactions)
df_conv = pd.DataFrame(te_array, columns=te.columns_)

print(f"상품 수: {len(te.columns_)}개")
print(f"상품 목록: {list(te.columns_)}")

In [ ]:
# Apriori 실행
frequent_conv = apriori(
    df_conv,
    min_support=0.1,  # 10% 이상
    use_colnames=True
)

print(f"빈발 항목집합: {len(frequent_conv)}개")
print("\n상위 빈발 항목:")
print(frequent_conv.sort_values('support', ascending=False).head(10))

In [ ]:
# 연관 규칙 생성
rules_conv = association_rules(
    frequent_conv,
    metric="lift",
    min_threshold=1.0  # Lift >= 1 (양의 상관관계만)
)

print(f"생성된 연관 규칙: {len(rules_conv)}개")

# 상위 규칙
print("\n상위 연관 규칙 (Lift 기준):")
top_rules = rules_conv.nlargest(10, 'lift')[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print(top_rules)

---

# Part 2: 연관 분석 심화 및 응용

---

## 2.1 추가 지표 이해

### Conviction (확신도)

$$Conviction(A \rightarrow B) = \frac{1 - Support(B)}{1 - Confidence(A \rightarrow B)}$$

- **해석**: 규칙이 틀릴 확률의 비율
- **높을수록**: 규칙이 더 신뢰할 만함

### Leverage (레버리지)

$$Leverage(A \rightarrow B) = Support(A \cup B) - Support(A) \times Support(B)$$

- **해석**: 독립일 때 대비 추가 발생 확률
- **0보다 크면**: 양의 상관관계

### Zhang's Metric

- **범위**: -1 ~ 1
- **해석**: 상관관계의 방향과 강도 동시 측정

In [ ]:
# 모든 지표 확인
print("연관 규칙 전체 지표:")
print(rules_conv.columns.tolist())

In [ ]:
# 지표별 상위 규칙 비교
print("지표별 상위 규칙 비교")
print("=" * 70)

metrics = ['lift', 'confidence', 'conviction', 'leverage']

for metric in metrics:
    if metric in rules_conv.columns:
        top_rule = rules_conv.nlargest(1, metric).iloc[0]
        print(f"\n[{metric.upper()} 최고 규칙]")
        print(f"  {list(top_rule['antecedents'])} → {list(top_rule['consequents'])}")
        print(f"  {metric}: {top_rule[metric]:.3f}")

---

## 2.2 Plotly 시각화

In [ ]:
# 시각화용 데이터 준비
rules_conv['antecedents_str'] = rules_conv['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_conv['consequents_str'] = rules_conv['consequents'].apply(lambda x: ', '.join(list(x)))
rules_conv['rule_str'] = rules_conv['antecedents_str'] + ' → ' + rules_conv['consequents_str']

In [ ]:
# 1. 산점도: Support vs Confidence (크기=Lift)
fig = px.scatter(
    rules_conv,
    x='support',
    y='confidence',
    size='lift',
    color='lift',
    hover_data=['rule_str'],
    title='연관 규칙 시각화: Support vs Confidence (크기/색상=Lift)',
    color_continuous_scale='RdYlGn'
)

fig.update_layout(
    width=800,
    height=600,
    xaxis_title='지지도 (Support)',
    yaxis_title='신뢰도 (Confidence)'
)

fig.show()

In [ ]:
# 2. 상위 규칙 막대 차트
top_rules_lift = rules_conv.nlargest(15, 'lift')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_rules_lift['lift'],
    y=top_rules_lift['rule_str'],
    orientation='h',
    marker_color='#3498db',
    text=top_rules_lift['lift'].round(2),
    textposition='outside'
))

fig.update_layout(
    title='상위 15개 연관 규칙 (Lift 기준)',
    xaxis_title='향상도 (Lift)',
    yaxis_title='규칙',
    height=600,
    width=900,
    yaxis=dict(autorange='reversed')
)

fig.show()

In [ ]:
# 3. 상품별 연관 빈도 히트맵
# 상위 상품만 선택
top_items = item_freq.head(8).index.tolist()

# 공출현 행렬 생성
cooccurrence = pd.DataFrame(0, index=top_items, columns=top_items)

for trans in convenience_store_transactions:
    for item1 in trans:
        for item2 in trans:
            if item1 in top_items and item2 in top_items:
                cooccurrence.loc[item1, item2] += 1

# 히트맵
fig = px.imshow(
    cooccurrence,
    labels=dict(x='상품', y='상품', color='공출현 횟수'),
    title='상품 공출현 히트맵',
    color_continuous_scale='Blues'
)

fig.update_layout(width=700, height=600)
fig.show()

---

## 2.3 추천 시스템 함수 구현

In [ ]:
def recommend_products(selected_items, rules_df, top_n=5, metric='lift'):
    """
    선택한 상품 기반 추천
    
    Parameters:
    - selected_items: 선택한 상품 (문자열 또는 리스트)
    - rules_df: 연관 규칙 DataFrame
    - top_n: 추천 개수
    - metric: 정렬 기준 ('lift', 'confidence', 'support')
    
    Returns:
    - DataFrame: 추천 결과
    """
    # 입력 처리
    if isinstance(selected_items, str):
        selected_items = [selected_items]
    selected_set = set(selected_items)
    
    # 선택한 상품이 전제에 포함된 규칙 필터링
    filtered = rules_df[
        rules_df['antecedents'].apply(lambda x: selected_set.issubset(x) or len(selected_set & x) > 0)
    ]
    
    if len(filtered) == 0:
        return pd.DataFrame({'message': ['추천할 상품이 없습니다.']})
    
    # 메트릭 기준 정렬
    result = filtered.nlargest(top_n, metric)[['consequents_str', 'confidence', 'lift', 'support']]
    result.columns = ['추천 상품', '신뢰도', '향상도', '지지도']
    result = result.reset_index(drop=True)
    result.index = result.index + 1
    
    return result


# 테스트
print("'삼각김밥'을 선택한 고객에게 추천:")
print(recommend_products('삼각김밥', rules_conv))

print("\n'맥주'를 선택한 고객에게 추천:")
print(recommend_products('맥주', rules_conv))

In [ ]:
# 복수 상품 추천
print("'삼각김밥', '콜라'를 선택한 고객에게 추천:")
print(recommend_products(['삼각김밥', '콜라'], rules_conv))

---

## 2.4 Gradio UI 인터랙티브 데모

In [ ]:
# Gradio 설치 (최초 1회)
# !pip install gradio

In [ ]:
try:
    import gradio as gr
    gradio_available = True
    print("Gradio 로드 완료")
except ImportError:
    gradio_available = False
    print("Gradio 미설치. pip install gradio 로 설치하세요.")

In [ ]:
if gradio_available:
    # 상품 목록
    product_list = sorted(list(te.columns_))
    
    def gradio_recommend(selected_item, metric, top_n):
        """
        Gradio용 추천 함수
        """
        result = recommend_products(selected_item, rules_conv, int(top_n), metric)
        return result
    
    # Gradio 인터페이스
    demo = gr.Interface(
        fn=gradio_recommend,
        inputs=[
            gr.Dropdown(choices=product_list, label="상품 선택"),
            gr.Radio(choices=['lift', 'confidence', 'support'], value='lift', label="정렬 기준"),
            gr.Slider(minimum=1, maximum=10, value=5, step=1, label="추천 개수")
        ],
        outputs=gr.Dataframe(label="추천 결과"),
        title="편의점 상품 추천 시스템",
        description="상품을 선택하면 함께 구매하면 좋은 상품을 추천해드립니다."
    )
    
    # 노트북에서 실행 (share=False)
    # demo.launch(share=False)
    print("Gradio 데모가 준비되었습니다.")
    print("실행하려면 위 셀의 주석(#)을 제거하고 demo.launch()를 실행하세요.")
else:
    print("Gradio가 설치되지 않아 데모를 건너뜁니다.")

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 지지도 계산하기 ⭐

**문제**: 다음 거래 데이터에서 '사과'의 지지도를 계산하세요.

```python
transactions = [
    ['사과', '바나나'],
    ['사과', '오렌지', '바나나'],
    ['오렌지', '포도'],
    ['사과', '포도'],
    ['바나나', '포도']
]
```

**기대 결과**: 0.6 (또는 60%)

In [ ]:
transactions = [
    ['사과', '바나나'],
    ['사과', '오렌지', '바나나'],
    ['오렌지', '포도'],
    ['사과', '포도'],
    ['바나나', '포도']
]

# 여기에 코드를 작성하세요

### Q2. 신뢰도와 향상도 계산하기 ⭐⭐

**문제**: Q1의 데이터에서 {사과} → {바나나} 규칙의 신뢰도와 향상도를 계산하세요.

**힌트**:
- 사과 포함 거래: 3개
- 사과+바나나 포함 거래: 2개
- 바나나 포함 거래: 3개

In [ ]:
# 여기에 코드를 작성하세요

### Q3. TransactionEncoder 사용하기 ⭐⭐

**문제**: Q1의 거래 데이터를 TransactionEncoder로 변환하세요.

**기대 결과**: 5x4 이진 행렬 DataFrame

In [ ]:
# 여기에 코드를 작성하세요

### Q4. Apriori 알고리즘 실행하기 ⭐⭐⭐

**문제**: Q3의 데이터에 apriori 알고리즘을 적용하여 빈발 항목집합을 찾으세요.

**요구사항**:
- min_support=0.3
- 결과를 support 기준 내림차순 정렬

In [ ]:
# 여기에 코드를 작성하세요

### Q5. 연관 규칙 생성 및 필터링 ⭐⭐⭐

**문제**: Q4의 빈발 항목집합에서 연관 규칙을 생성하고, Lift > 1인 규칙만 필터링하세요.

**기대 결과**: Lift > 1인 규칙 목록

In [ ]:
# 여기에 코드를 작성하세요

### Q6. 규칙 시각화 (산점도) ⭐⭐⭐

**문제**: 편의점 데이터(`rules_conv`)를 사용하여 Support vs Confidence 산점도를 그리세요.

**요구사항**:
- x축: support, y축: confidence
- 마커 크기: lift 비례
- 마커 색상: lift 색상 스케일

In [ ]:
# 여기에 코드를 작성하세요

### Q7. 추천 함수 구현하기 ⭐⭐⭐⭐

**문제**: 특정 상품을 입력받아 연관 상품을 추천하는 함수를 작성하세요.

**요구사항**:
- 입력: 상품명(문자열), 추천 개수
- 출력: 추천 상품 리스트 (confidence 기준 정렬)

In [ ]:
def my_recommend(product, n=5):
    """
    상품 추천 함수
    """
    # 여기에 코드를 작성하세요
    pass

### Q8. 상위 규칙 막대 차트 ⭐⭐⭐⭐

**문제**: Confidence 상위 10개 규칙을 수평 막대 차트로 시각화하세요.

**요구사항**:
- y축: 규칙 문자열 (예: "삼각김밥 → 컵라면")
- x축: confidence 값
- 색상: lift에 따라 그라데이션

In [ ]:
# 여기에 코드를 작성하세요

### Q9. 연관 분석 리포트 생성 ⭐⭐⭐⭐⭐

**문제**: 편의점 데이터 연관 분석 결과를 요약하는 리포트 함수를 작성하세요.

**출력 항목**:
1. 총 규칙 수
2. 평균 지지도, 신뢰도, 향상도
3. Lift > 2인 강력한 규칙 목록
4. 가장 많이 등장하는 결론(consequent) 상품 Top 5

In [ ]:
def generate_association_report(rules_df):
    """
    연관 분석 리포트 생성
    """
    # 여기에 코드를 작성하세요
    pass

### Q10. 종합 대시보드 ⭐⭐⭐⭐⭐

**문제**: 연관 분석 결과를 종합 대시보드로 시각화하세요.

**분석 항목** (2x2 레이아웃):
1. 상품별 등장 빈도 (막대 차트)
2. Support vs Confidence (산점도)
3. 상위 규칙 (수평 막대)
4. 지표 분포 (히스토그램)

**요구사항**:
- `make_subplots` 사용
- 인사이트 3개 이상 도출

In [ ]:
# 여기에 코드를 작성하세요

---

## 📊 학습 정리

### Part 1: 연관 규칙 기초 핵심 요약

| 개념 | 수식 | 해석 |
|------|------|------|
| 지지도 (Support) | P(A∩B) | 동시 발생 확률 |
| 신뢰도 (Confidence) | P(B\|A) | A 구매 시 B 구매 확률 |
| 향상도 (Lift) | P(A∩B) / P(A)P(B) | 독립 대비 상관관계 |

### Part 2: 심화 및 응용 핵심 요약

| 도구 | 함수 | 활용 |
|------|------|------|
| TransactionEncoder | `fit_transform()` | 거래 → 이진 행렬 |
| Apriori | `apriori(min_support)` | 빈발 항목 탐색 |
| 연관 규칙 | `association_rules(metric)` | 규칙 생성 |
| 시각화 | `px.scatter()` | Support-Confidence 산점도 |
| UI | `gr.Interface()` | 인터랙티브 추천 |

### 💡 실무 팁

1. **min_support 튜닝**: 너무 낮으면 노이즈, 너무 높으면 규칙 없음 (0.01~0.1 권장)
2. **Lift > 1 필터**: 양의 상관관계만 의미 있음
3. **Confidence 해석**: "A를 사면 B도 산다" 확률
4. **Conviction 활용**: 규칙의 방향성 평가에 유용
5. **시각화 필수**: 산점도로 전체 규칙 분포 파악
6. **실시간 추천**: 장바구니에 상품 담을 때마다 추천 갱신
7. **A/B 테스트**: 추천 효과를 매출로 검증